In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [2]:
d1 = pd.read_csv('/config/workspace/house_rent.csv')

In [3]:
d1.head()

,House_Age,METRO,REGION,BEDROOMS,ROOMS,PERSONS,Rent
0,82,Town,Estern,2,6,1,3146.450682
1,50,Others,Northern,4,6,4,5524.610550
2,53,Others,Northern,4,7,2,5605.127558
3,67,Others,Northern,3,6,2,7411.836744
4,50,Central City,Northern,3,6,3,9567.554434


In [5]:
X = d1.drop(columns=['Rent'],axis=1)

In [6]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [7]:
X = preprocessor.fit_transform(X)

In [8]:
y = d1['Rent']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((29340, 12), (7335, 12))

In [10]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [12]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 8417.8519
- Mean Absolute Error: 4543.3190
- R2 Score: 0.1942
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7861.1798
- Mean Absolute Error: 4345.3483
- R2 Score: 0.2085


Lasso
Model performance for Training set
- Root Mean Squared Error: 8417.6953
- Mean Absolute Error: 4541.7090
- R2 Score: 0.1943
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7861.1975
- Mean Absolute Error: 4343.6775
- R2 Score: 0.2085


Ridge
Model performance for Training set
- Root Mean Squared Error: 8417.6932
- Mean Absolute Error: 4542.3431
- R2 Score: 0.1943
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7861.3485
- Mean Absolute Error: 4344.3038
- R2 Score: 0.2085


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 7315.0119
- Mean Absolute Error: 4008.3024


In [ ]:
print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

In [13]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,CatBoosting Regressor,0.237062
6,XGBRegressor,0.218553
0,Linear Regression,0.208544
1,Lasso,0.208540
2,Ridge,0.208510
3,K-Neighbors Regressor,0.098573
5,Random Forest Regressor,0.023267
8,AdaBoost Regressor,-0.224933
4,Decision Tree,-0.559339
